In [3]:
import csv
import os
import re

def courtHistory(listHasil):

	csvFolrder = "./"
	csvName = "courtHistory.csv"

	if os.path.exists(csvName):

		f = open(csvName, 'a', newline='\n')
		print(f)
		print("ada")
		w = csv.writer(f)

	else:

		f = open(csvName, 'w', newline='\n')
		print(f)
		print("tidak ada")
    
		w = csv.writer(f)
		w.writerow(('Nomor Putusan', 'Nama Terdakwa 1', 'Nama Terdakwa 2', 'Nama Terdakwa 3', 'Tuntutan Pidana', 'Tuntutan KUHP', 'Tuntukan Hukuman', 'Putusan Pidana', 'Putusan Hukuman', 'Tanggal Putusan', 'Hakim Ketua', 'Hakim Anggota1', 'Hakim Anggota2', 'Panitera', 'Penuntut Umum'))
    
	# menulis file csv
	for s in listHasil:
		w.writerow(s)

	f.close()
	berhasil = "\nCreate Csv file Berhasil\n" 
	return berhasil





In [4]:
import re

def generateEntity(pathFile,fileName):

    print("a")
    folderData = pathFile
    folderHasil = "./OUTPUT/"
    #namaFile = "12Pid.B2015.PNJktTim.txt"
    namaFile = fileName
    namaFileHasil = "O-"+namaFile
    listHasil =[]

    file_putusan = open(folderData+namaFile, "r", encoding='UTF8')
    baca_baris = file_putusan.readlines()

    terdakwa = 0
    tuntutan_pidana = 0
    tuntutanPidana = 0
    tuntutanHukuman = 0
    tindakPidana = 0
    hukuman = 0
    nomor_putusan = 0
    listTerdakwa=[]

    print("")
    print("====== KEPUTUSAN PENGADILAN =====")
    with open(folderHasil + namaFile, "w", encoding="utf-8") as file:

        for baris in baca_baris:
            panjang_baris = len(baris)
            if panjang_baris > 3:
                baris=baris.lower()

                # mendapatkan entitas NOMOR PUTUSAN
                if baris.find("pid.b") >= 0 and nomor_putusan == 0:
                    nomor_putusan +=1
                    eNomor = re.search( r'\d+/(.)*/\d{4}/(.)*',baris, re.M|re.I)	

                    print("Nomor Putusan : " + eNomor.group())
                    file.writelines("Nomor Putusan : " + eNomor.group() + "\n")
                    listHasil.append(eNomor.group().rstrip("\n")) 
                    
                # mendapatkan entitas NAMA TERDAKWA
                if baris.find("nama terdakwa :") >= 0 or baris.find("nama lengkap :") >= 0:
                    terdakwa += 1
                    eTerdakwa = re.search( r'nama (lengkap|terdakwa) :(.*)',baris, re.M|re.I)                
                    terdakwaKe = str(terdakwa)
                    print("Terdakwa Ke : "+terdakwaKe)
                    print("Nama Terdakwa : "+eTerdakwa.group(2))
                    file.writelines("Terdakwa Ke : "+terdakwaKe + "\n")
                    file.writelines("Nama Terdakwa : "+eTerdakwa.group(2) + "\n" + "\n")
                    listTerdakwa.append(eTerdakwa.group(2).rstrip("\n"))

                if (baris.find("terdakwa ditahan") >= 0 or baris.find("telah ditahan") >= 0 or baris.find("terdakwa berada") >= 0 )and terdakwa >= 1:
                    listHasil.append(listTerdakwa[0])
                    if terdakwa == 1: 
                        listHasil.append("")                
                        listHasil.append("")
                    if terdakwa == 2: 
                        listHasil.append(listTerdakwa[1])                
                        listHasil.append("")
                    if terdakwa == 3: 
                        listHasil.append(listTerdakwa[1])                
                        listHasil.append(listTerdakwa[2])
                    terdakwa = 0
                    
                # mendapatkan entitas TUNTUTAN PIDANA
                if baris.find("menyatakan terdakwa") >= 0 and tuntutan_pidana == 0 and tuntutanPidana==0:
                    print("====== TUNTUTAN =====")
                    file.writelines("TUNTUTAN" + "\n")
                    if baris.find("sebagaimana") >= 0:
                        ePidana = re.search( r'melakukan tindak pidana (.*) sebagaimana',baris, re.M|re.I)                
                    else:
                        ePidana = re.search( r'melakukan tindak pidana (.*) ;',baris, re.M|re.I)
                        
                    if ePidana:
                        print("Tindak Pidana : "+ePidana.group(1))    
                        file.writelines("Tindak Pidana : "+ePidana.group(1) + "\n")        
                        listHasil.append(ePidana.group(1))
                    else:
                        print("Tindak Pidana : ")            
                        listHasil.append("")                

                    eKUHP = re.search( r'pasal (.*) kuhp',baris, re.M|re.I)
                    if eKUHP: 
                        print("Melanggar KUHP : " +eKUHP.group())
                        file.writelines("Melanggar KUHP : " +eKUHP.group() + "\n")
                        listHasil.append(eKUHP.group())
                    else:
                        print("Melanggar KUHP : ")
                        file.writelines("Melanggar KUHP : " + "\n")
                        listHasil.append("")                    
                    tuntutanPidana = 1

                if baris.find("menjatuhkan pidana") >= 0 and tuntutan_pidana == 0 and tuntutanHukuman == 0:
                    #print(baris)
                    etuntutan = re.search( r'selama (.*) dikurangi',baris, re.M|re.I)
                    print("Tuntutan Hukuman : "+etuntutan.group(1))
                    file.writelines("Tuntutan Hukuman : "+etuntutan.group(1) + "\n")
                    listHasil.append(etuntutan.group(1))                
                    tuntutan_pidana = 1

                # mendapatkan entitas PUTUSAN PIDANA
                if baris.find("menyatakan terdakwa") >= 0 and baris.find("terbukti") >= 0 and tuntutan_pidana == 1 and tuntutanPidana == 1:
                    print("====== PUTUSAN =====")
                    print(baris)
                    if baris.find("sebagaimana") >= 0:
                        ePutPidana = re.search( r'melakukan tindak pidana (.*) sebagaimana',baris, re.M|re.I)
                    else:
                        ePutPidana = re.search( r'melakukan tindak pidana (.*) ;',baris, re.M|re.I)
                    print("Putusan Pidana : "+ePutPidana.group(1))        
                    file.writelines("Putusan Pidana : "+ePutPidana.group(1))    
                    listHasil.append(ePutPidana.group(1))
                    tuntutanHukuman = 1

                if baris.find("menjatuhkan pidana") >= 0 and tuntutan_pidana == 1 and tuntutanHukuman == 1:
                    #print(baris)
                    if baris.find("dikurangi") >= 0:
                        ePutusan = re.search( r'selama (.*) dikurangi',baris, re.M|re.I)
                    else:
                        ePutusan = re.search( r'selama (.*) ;',baris, re.M|re.I)    
                    print("Putusan Hukuman : "+ePutusan.group(1))
                    file.writelines("Putusan Hukuman : "+ePutusan.group(1))
                    listHasil.append(ePutusan.group(1))
                    #tuntutan_pidana = 1    
            
                # mendapatkan HAKIM KETUA MAJLIS HAKIM            
                if baris.find("diputuskan dalam") >= 0:
                    print("====== PENUTUP =====")
                    cari_tgl_putusan = baris.find("tanggal")
                    cari_tgl_putusan1 = baris.find("oleh")

                    if baris.find(", se") >= 0:
                        cari_hakim_ketua1 = baris.find(", se")
                    else:
                        cari_hakim_ketua1 = baris.find(". se")

                    if baris.find("oleh kami") >= 0:
                        oleh = 9
                    else:
                        oleh = 5

                    if baris.find("ketua majelis hakim, ") >= 0:  
                        cari_hakim_anggota1 = baris.find("ketua majelis hakim, ")+21
                    elif baris.find("hakim ketua majelis, ") >=0:    
                        cari_hakim_anggota1 = baris.find("hakim ketua majelis, ")+21
                    elif baris.find("hakim ketua, ") >=0:
                        cari_hakim_anggota1 = baris.find("hakim ketua, ")+13

                    cari_hakim_anggota2 = baris.find("masing")            
                    hakim_anggota = baris[cari_hakim_anggota1:cari_hakim_anggota2-1]
                    hakim_anggota1 = hakim_anggota[0:hakim_anggota.find(" dan")]
                    hakim_anggota2 = hakim_anggota[hakim_anggota.find(" dan")+5:len(hakim_anggota)]

                    if baris.find("dibantu oleh") >= 0:
                        cari_panitera1 = baris.find("dibantu oleh")+13
                    else:    
                        cari_panitera1 = baris.find("dibantu")+8

                    if baris.find("sebagai panitera pengganti") >= 0:              
                        cari_panitera2 = baris.find("sebagai panitera pengganti")
                    elif baris.find("panitera pengganti") >= 0:              
                        cari_panitera2 = baris.find("panitera pengganti")
                        
                    if baris.find("dihadiri oleh") >= 0:
                        cari_penuntut_umum1 = baris.find("dihadiri oleh")+13
                    elif baris.find("dihadiri") >= 0:
                        cari_penuntut_umum1 = baris.find("dihadiri")+9    

                    if baris.find("jaksa penuntut umum") >= 0:
                        cari_penuntut_umum2 = baris.find("jaksa penuntut umum")
                    elif baris.find("penuntut umum") >= 0:
                        cari_penuntut_umum2 = baris.find("penuntut umum")    
                    
                    tanggalPutusan = baris[cari_tgl_putusan+8:cari_tgl_putusan1]
                    hakimKetua = baris[cari_tgl_putusan1+oleh:cari_hakim_ketua1]
                    hakimAnggota1 = hakim_anggota1
                    hakimAnggota2 = hakim_anggota2
                    panitera = baris[cari_panitera1:cari_panitera2]
                    penuntutUmum = baris[cari_penuntut_umum1:cari_penuntut_umum2]

                    print("Tanggal Putusan : "+tanggalPutusan)
                    print("Hakim Ketua : "+hakimKetua)
                    print("Hakim Anggota 1 : "+hakimAnggota1)
                    print("Hakim Anggota 2 : "+hakimAnggota2)
                    print("Panitera : "+panitera)
                    print("Penuntut Umum : "+penuntutUmum)
                    
                    file.writelines("\n" + "\n" + "PUTUSAN" + "\n")
                    file.writelines("Tanggal Putusan : " + tanggalPutusan + "\n")
                    file.writelines("Hakim Ketua : " + hakimKetua + "\n")
                    file.writelines("Hakim Anggota 1 : "+hakimAnggota1 + "\n")
                    file.writelines("Hakim Anggota 2 : "+hakimAnggota2 + "\n")
                    file.writelines("Panitera : "+panitera + "\n")
                    file.writelines("Penuntut Umum : "+penuntutUmum + "\n")

                    listHasil.append(tanggalPutusan)
                    listHasil.append(hakimKetua)
                    listHasil.append(hakimAnggota1)
                    listHasil.append(hakimAnggota2)
                    listHasil.append(panitera)
                    listHasil.append(penuntutUmum)
        
            
            
    file_putusan.close()
    print("======================")
    return listHasil


# #start_path = "./INPUT/"
# #namaFile = "48Pid.B2015.PNJKTTIM.txt"

# #hasil = generateEntity(start_path, namaFile)
# #print(hasil)


In [5]:
import re

state_data = {
    0: "tidak ada",
    1: "terdakwa"
}

In [6]:
def cari_terdakwa(data):
    terdakwa = 1
    hasil_terdakwa = []
    data_arr = data.lower().split("\n")
    index_terdakwa = []
    
    if re.search("terdakwa " + str(terdakwa), data.lower()):
        while re.search("terdakwa " + str(terdakwa), data.lower()):
            terdakwa += 1
        
        terdakwa = 1
        for baris in data_arr:
            if re.search(r"nama (lengkap|terdakwa) :(.*)", baris):
                print(baris.split(":")[1][0:(len(baris.split(":")[1]) - 1)])
        
        
        
def generateEntity(pathFile,fileName):
    
    folderData = pathFile
    folderHasil = "./OUTPUT/"
    #namaFile = "12Pid.B2015.PNJktTim.txt"
    namaFile = fileName
    namaFileHasil = "O-"+namaFile
    listHasil = []
    
    state = state_data[0]
    
    file_putusan = open(folderData+namaFile, "r", encoding='UTF8')
    baca_baris = file_putusan.read()
    cari_terdakwa(baca_baris)
    
    # for baris in baca_baris:
    
    #     # print(baris.split("\n"))
    #     if baris.split("\n")[0] == "berikut dalam perkara Terdakwa :":
    #         if re.search("perkara terdakwa", baris.split("\n")[0].lower()):
    #             state = state_data[1]
    
    #         if state == state_data[1]:
    #             print("data")            
        
        # print(re.search("perkara terdakwa", baris.lower()))


# generateEntity("./INPUT/", "output2.txt")
    

In [7]:
import os
# from courtHistoryCsv import courtHistory

def main():

    start_path = "./input/"
    hasil_path = "./output/"
    daftarFile = os.listdir(start_path)
    listHasil = []

    # namaFile = "48Pid.B2015.PNJKTTIM.txt"
    # namaFile = "161Pid.B2015.PNJktTim.txt"
    # hasil = generateEntity(start_path, namaFile)
    # listHasil.append(hasil)

    for tiapFile in daftarFile:
        # print(tiapFile)
        hasil = generateEntity(start_path, tiapFile)
        listHasil.append(hasil)
        print("======= ROW HASIL =======")    
        # print(hasil)
        break


	
    print("======= HASIL AKHIR =======")
 

main()


======= ROW HASIL =======
======= HASIL AKHIR =======


In [8]:
def preproses(data):
    return data.lower().split("\n")

In [9]:
def exctract_terdakwa(data, finder):
    data = data[data.find(finder):]
    nama_raw = data[data.find("nama lengkap") + len("nama lengkap : "):]
    nama_raw = nama_raw[:nama_raw.find("2")].replace(";", "")
    return nama_raw.strip()

def cari_terdakwa(data):
    data = " ".join(" ".join(preproses(data)).split())
    total_terdakwa = 0
    list_terdakwa = []
    while data.find("terdakwa " + str(total_terdakwa + 1)) != -1:
        total_terdakwa += 1
    if total_terdakwa < 1:
        list_terdakwa.append(exctract_terdakwa(data, "perkara terdakwa"))
    else:
        for i in range(1, total_terdakwa + 1):
            list_terdakwa.append((exctract_terdakwa(data, "terdakwa " + str(i))))
    
    return list_terdakwa
            
list_file = os.listdir("./INPUT/")

# for filename in list_file:
#     with open("./INPUT/" + filename, "r", encoding="utf-8") as file:
#         print(cari_terdakwa(file.read()))
    

In [10]:
def cari_tuntutan_tp(data):

    data = " ".join(data.lower().split())

    data = data[data.find("melakukan"):] + " "
    data = data[:data.find("sebagaimana")] + " "
    data = data[:data.find("sesuai")] + " "
    
    return " ".join(data.split()[3:]).replace("“", "").replace("”", "").replace("\"", "").strip()


list_file = os.listdir("./INPUT/")

# for filename in list_file:
#     data = ""
    
#     with open("./INPUT/" + filename, "r", encoding="utf-8") as file:
#         data = file.read()

#     # print(filename)
#     (cari_tuntutan_tp(data))
#     print()
#     # print()

In [11]:
def cari_putusan_tp(data):
    data = " ".join(data.split())
    start = data.find("MENGADILI:") if data.find("MENGADILI:") != -1 else data.find("M E N G A D I L I:")
    data = data[start:]
    end = data.find("2.")
    data = data[:end]
    
    data = " ".join(data.lower().split())

    data = data[data.find("melakukan"):] + " "
    data = data[:data.find("sebagaimana")] + " "
    data = data[:data.find(" s ebagaimana")] + " "
    data = data[:data.find("sesuai")] + " "
    
    return " ".join(data.split()[3:]).replace("“", "").replace("”", "").replace("\"", "").strip()

# for filename in list_file:
#     data = ""
    
#     with open("./INPUT/" + filename, "r", encoding="utf-8") as file:
#         data = file.read()
#         print(cari_putusan_tp(data))
    
        
    
    

In [12]:
def cari_kuhp_tp(data):
    data = " ".join(data.split()).lower()
    start = data.find("setelah mendengar pembacaan tuntutan pidana yang diajukan oleh penuntut umum yang pada pokoknya sebagai berikut:")
    data = data[start:]
    
    pattern = [
        "pasal",
        "sebagaimana diatur dan diancam dalam",
    ]
    
    start = -1
    
    for pat in pattern:
        if data.find(pat) != -1:
            data = data[data.find(pat):] + " "
            data = data[:data.find("2.")] + " "
            data = data[:data.find(";")] + " "
            data = data[:data.find("tentang")] + " "
            data = data.strip()
            break

    
    return data

    
# for filename in list_file:
#     data = ""
#     with open("./INPUT/" + filename, "r", encoding="utf-8") as file:
#         data = file.read()
#         print(cari_kuhp_tp(data))

In [13]:
def cari_hukuman_tp(data):
    data = " ".join(data.split()).lower()
    start = data.find("setelah mendengar pembacaan tuntutan pidana yang diajukan oleh penuntut umum yang pada pokoknya sebagai berikut:")
    data = data[start:]
    
    start = data.find("selama")
    end = data.find("3.")
    hasil = data[start:end].strip()
  
    if hasil[-1] in ".;":
        hasil = hasil[:-1]
        
    return hasil

# for filename in list_file:
#     data = ""
#     with open("./INPUT/" + filename, "r", encoding="utf-8") as file:
#         data = file.read()
#         print(cari_hukuman_tp(data))

In [18]:
def cari_hukuman_p(data):
    
    data = " ".join(data.split())
    start = data.find("MENGADILI:") if data.find("MENGADILI:") != -1 else (data.find("M E N G A D I L I:") if data.find("M E N G A D I L I:") != -1 else data.find("M E N G A D I L I :"))
    data = data[start:]
    
    start = data.find("selama")
    end = data.find("3.")
    print(start)
    hasil = data[start:end].strip()
    

    if hasil[-1] in ".;":
        hasil = hasil[:-1]
        
    return hasil

for filename in list_file:
    data = ""
    with open("./input/" + filename, "r", encoding="utf-8") as file:
        data = file.read()
        print(cari_hukuman_p(data))

366
selama 7 (tujuh) bulan
353
selama 8 (delapan) bulan
379
selama 7 (tujuh) bulan dan 20 (dua puluh) hari
402
selama 9 (sembilan) bulan
267
selama 5 (lima) bulan
1333
selama 2 (dua) tahun
319
selama 1 (satu) tahun dan 10 (sepuluh) bulan
285
selama 1 (satu) tahun dan 6 (enam) bulan
-1


IndexError: string index out of range

In [ ]:
def cari_hakim(data):
    data = " ".join(data.split())
    start = data.find("MENGADILI:") if data.find("MENGADILI:") != -1 else data.find("M E N G A D I L I:")
    data = data[start:].lower()
    data = data[data.find("6."):]
    data = data[data.find("oleh kami") + len("oleh kami") if data.find("oleh kami") != -1 else data.find("oleh") + len("oleh"):].strip()
    if data[0] == ",":
        data = data[2:]
    
    # print(data)

    
    data_hakim = {
        
    }
    
    data_hakim["hakim ketua"] = data[:data.find(", sebagai hakim")]
    
    hakim_anggota = data[data.find("sebagai hakim ketua") + len("sebagai hakim ketua"):data.find("masing")].strip().split("dan")
    for hakim in range(len(hakim_anggota)):
        if hakim_anggota[hakim][0] == ",":
            hakim_anggota[hakim] = hakim_anggota[hakim][1:].strip()
        
        if hakim_anggota[hakim][-1] == ",":
            hakim_anggota[hakim] = hakim_anggota[hakim][:-1].strip()
        
            
    data_hakim["hakim anggota"] = hakim_anggota
    return data_hakim
    
# for filename in list_file:
#     data = ""
#     with open("./INPUT/" + filename, "r", encoding="utf-8") as file:
#         data = file.read()
#         print(cari_hakim(data))

In [ ]:
def cari_tanggal_putusan(data):
    data = " ".join(data.split())
    start = data.find("MENGADILI:") if data.find("MENGADILI:") != -1 else data.find("M E N G A D I L I:")
    data = data[start:]
    start = data.find("6.")
    data = data[start:]
    data = data.lower()
    data = data[data.find("tanggal") + len("tanggal"):]
    data = data[:data.find(",")].strip()
    return data
    
# for filename in list_file:
#     data = ""
#     with open("./INPUT/" + filename, "r", encoding="utf-8") as file:
#         data = file.read()
#         print(cari_tanggal_putusan(data))

In [ ]:
def cari_panitera(data):
    data = " ".join(data.split())
    start = data.find("MENGADILI:") if data.find("MENGADILI:") != -1 else data.find("M E N G A D I L I:")
    data = data[start:]
    start = data.find("6.")
    data = data[start:]
    data = data.lower()
    data = data[data.find("dibantu oleh") + len("dibantu oleh "):]
    data = data[:data.find("panitera")].strip()
    return data

# for filename in list_file:
#     data = ""
#     with open("./INPUT/" + filename, "r", encoding="utf-8") as file:
#         data = file.read()
#         print(cari_panitera(data))

In [ ]:
def cari_penuntut(data):
    data = " ".join(data.split())
    start = data.find("MENGADILI:") if data.find("MENGADILI:") != -1 else data.find("M E N G A D I L I:")
    data = data[start:]
    start = data.find("6.")
    data = data[start:]
    data = data.lower()
    data = data[data.find("dihadiri oleh") + len("dihadiri oleh "):]
    data = data[:data.find("penuntut")].strip()
    
    return data

# for filename in list_file:
#     data = ""
#     with open("./INPUT/" + filename, "r", encoding="utf-8") as file:
#         data = file.read()
#         print(cari_penuntut(data))

In [ ]:
def cari_nomor_putusan(data):
    start_index = data.find("Nomor")
    end_index = data.find("DEMI")
    return data[start_index + len("Nomor"):end_index].strip()

# for filename in list_file:
#     data = ""
#     with open("./INPUT/" + filename, "r", encoding="utf-8") as file:
#         data = file.read()
#         print(cari_nomor_putusan(data))

In [ ]:
def generateEntity(fileName):
    input_folder = "./input/"
    output_folder = "./output/"
    
    with open(output_folder + "O-" + fileName, "w", encoding="utf-8") as output_file:

        with open(input_folder + fileName, "r", encoding="utf-8") as input_file:
            data = input_file.read()
            nomor_putusan = cari_nomor_putusan(data)
            terdakwa = cari_terdakwa(data)
            
            tuntutan_tp = cari_tuntutan_tp(data)
            kuhp = cari_kuhp_tp(data)
            hukuman = cari_hukuman_tp(data)
            
            putusan_tp = cari_putusan_tp(data)
            hukuman_p = cari_hukuman_p(data)
            tanggal = cari_tanggal_putusan(data)
            hakim = cari_hakim(data)
            panitera = cari_panitera(data)
            penuntut = cari_penuntut(data)

            
            print("Nomor Putusan: " + nomor_putusan)
            output_file.writelines("Nomor Putusan: " + nomor_putusan + "\n")
            for terdakwa_index in range(len(terdakwa)):
                print("Terdakwa Ke: " + str(terdakwa_index + 1))
                output_file.writelines("Terdakwa Ke: " + str(terdakwa_index + 1) + "\n")
                print("Nama Terdakwa: " + terdakwa[terdakwa_index])
                output_file.writelines("Nama Terdakwa: " + terdakwa[terdakwa_index] + "\n" + "\n")
                print()

            print("TUNTUTAN")
            output_file.writelines("TUNTUTAN" + "\n")
            print("Tindak Pidana: " + tuntutan_tp )
            output_file.writelines("Tindak Pidana: " + tuntutan_tp + "\n")
            print("Melanggar KUHP: " + kuhp)
            output_file.writelines("Melanggar KUHP: " + kuhp + "\n")
            print("Tuntutan Pidana: " + hukuman)
            output_file.writelines("Tuntutan Pidana: " + hukuman + "\n" + "\n")
            
            print()
            print("PUTUSAN")
            output_file.writelines("PUTUSAN" + "\n")
            print("Tindak Pidana: " + putusan_tp )
            output_file.writelines("Tindak Pidana: " + putusan_tp + "\n")
            print("Hukunan Pidana: " + hukuman_p)
            output_file.writelines("Hukunan Pidana: " + hukuman_p + "\n")
            print("Tanggal Putusan: " + tanggal)
            output_file.writelines("Tanggal Putusan: " + tanggal + "\n"  + "\n")
            
            print()
            print("Hakim Ketua: " + hakim["hakim ketua"])
            output_file.writelines("Hakim Ketua: " + hakim["hakim ketua"] + "\n")
            
            for h in range(len(hakim["hakim anggota"])):
                print("Hakim Anggota " + str(h + 1) + ": ", hakim["hakim anggota"][h])
                output_file.writelines("Hakim Anggota " + str(h + 1) + ": " + hakim["hakim anggota"][h] + "\n")
                
            print("Panitera: " + panitera)
            output_file.writelines("Panitera: " + panitera + "\n")
            print("Penuntut Umum: " + penuntut)
            output_file.writelines("Penuntut Umum: " + penuntut + "\n")
            
         
    

In [ ]:
import os
# from courtHistoryCsv import courtHistory

def main():

    start_path = "./input/"
    hasil_path = "./output/"
    daftarFile = os.listdir(start_path)
    listHasil = []


    for tiapFile in daftarFile:
        print("======= ROW HASIL =======")    
        hasil = generateEntity(tiapFile)
        print("======= ROW AKHIR =======")
        print()
 

main()


======= ROW HASIL =======



IndexError: string index out of range

In [56]:
import re

def extract_detailed_information(text):
    info = {}

    # === Nomor Putusan ===
    nomor = re.search(r'(\d+/\w+/\d{4}/\w+)', text)  # contoh: 518/Pid.B/2024/PN
    if not nomor:
        nomor = re.search(r"Nomor\s*[:.]?\s*([\w./-]+)", text, re.IGNORECASE)
    info["Nomor Putusan"] = nomor.group(1).strip() if nomor else "Tidak ditemukan"

    # === Nama Terdakwa ===
    terdakwa = re.findall(r"Nama (?:lengkap|terdakwa)\s*:\s*(.*)", text, re.IGNORECASE)
    info["Nama Terdakwa"] = [t.strip() for t in terdakwa] if terdakwa else ["Tidak ditemukan"]

    # === Tindak Pidana ===
    tindak = re.search(r"melakukan tindak pidana\s+([^\n;]*)", text, re.IGNORECASE)
    if not tindak:
        tindak = re.search(r"Tindak Pidana\s*[:.]?\s*(.*)", text, re.IGNORECASE)
    info["Tindak Pidana"] = tindak.group(1).strip() if tindak else "Tidak ditemukan"

    # === Pasal KUHP ===
    pasal = re.search(r"pasal\s+([\d\w\s()., dan]+KUHP)", text, re.IGNORECASE)
    info["Melanggar KUHP"] = pasal.group(1).strip() if pasal else "Tidak ditemukan"

    # === Tuntutan Pidana (dari jaksa) ===
    tuntutan = re.search(r"menuntut\s+supaya\s+terdakwa.?pidana\s+(.)", text, re.IGNORECASE)
    if not tuntutan:
        tuntutan = re.search(r"tuntutan pidana\s*[:.]?\s*(.*)", text, re.IGNORECASE)
    info["Tuntutan Pidana"] = tuntutan.group(1).strip() if tuntutan else "Tidak ditemukan"

    # === Hukuman Pidana (amar putusan) ===
    putusan = re.search(r"menjatuhkan pidana.?selama\s+(.?)(?:;|\n)", text, re.IGNORECASE)
    if not putusan:
        putusan = re.search(r"pidana penjara\s+(.*?)(?:;|\n)", text, re.IGNORECASE)
    info["Hukuman Pidana"] = putusan.group(1).strip() if putusan else "Tidak ditemukan"

    # === Tanggal Putusan ===
    tanggal = re.search(r"tanggal\s+([\w\s\d]+?\d{4})", text, re.IGNORECASE)
    info["Tanggal Putusan"] = tanggal.group(1).strip() if tanggal else "Tidak ditemukan"

    # === Hakim Ketua ===
    hakim_ketua = re.search(r"Hakim Ketua\s*[:.]?\s*(.*)", text, re.IGNORECASE)
    if not hakim_ketua:
        hakim_ketua = re.search(r"oleh kami,?\s*(.*?),\s*sebagai Hakim Ketua", text, re.IGNORECASE)
    info["Hakim Ketua"] = hakim_ketua.group(1).strip() if hakim_ketua else "Tidak ditemukan"

    # === Hakim Anggota ===
    hakim_anggota = re.findall(r"Hakim Anggota\s*\d?\s*[:.]?\s*(.*)", text, re.IGNORECASE)
    info["Hakim Anggota"] = [h.strip() for h in hakim_anggota] if hakim_anggota else ["Tidak ditemukan"]

    # === Panitera ===
    panitera = re.search(r"Panitera\s*[:.]?\s*(.*)", text, re.IGNORECASE)
    if not panitera:
        panitera = re.search(r"dibantu oleh\s*(.*?),\s*Panitera Pengganti", text, re.IGNORECASE)
    info["Panitera"] = panitera.group(1).strip() if panitera else "Tidak ditemukan"

    # === Penuntut Umum ===
    penuntut = re.search(r"Penuntut Umum\s*[:.]?\s*(.*)", text, re.IGNORECASE)
    if not penuntut:
        penuntut = re.search(r"dihadiri oleh\s*(.*?),\s*Penuntut Umum", text, re.IGNORECASE)
    info["Penuntut Umum"] = penuntut.group(1).strip() if penuntut else "Tidak ditemukan"

    return info

In [ ]:
import re

def extract_detailed_information(text):
    info = {}
    text = " ".join(text.split())
    # === Nomor Putusan ===
    nomor = re.search(r"Nomor\s+([\w./-]+)", text, re.IGNORECASE)
    info["Nomor Putusan"] = nomor.group(1).strip() if nomor else "Tidak ditemukan"

    # === Nama Terdakwa ===
    terdakwa = re.findall(r"Nama lengkap\s*:\s*(.*)\s*2", text, re.IGNORECASE)
    info["Nama Terdakwa"] = [t[:t.find("2")].strip(" ;") for t in terdakwa] if terdakwa else ["Tidak ditemukan"]

    # === Tindak Pidana ===
    tindak =  re.search(r"melakukan\s+(?:beberapa\s+)?tindak\s+pidana\s+(.*?)(?=Pasal|Dakwaan|KUHP|;|\n\d|\Z)",text,re.IGNORECASE | re.DOTALL)
    if not tindak:
        tindak = re.search(r"Tindak Pidana\s*[:.]?\s*(.*)", text, re.IGNORECASE)
    info["Tindak Pidana"] = tindak.group(1).replace("\n", " ").strip(" ;") if tindak else "Tidak ditemukan"

    # === Pasal KUHP ===
    pasal = re.search(r"Pasal\s+([\d\w\s().,-]+KUHP)", text, re.IGNORECASE)
    info["Melanggar KUHP"] = pasal.group(1).strip(" ;") if pasal else "Tidak ditemukan"

    # === Tuntutan Pidana ===
    tuntutan = re.search( r"pidana\s+penjara.*?selama\s+(.*?)(?=,|;|\n\d|\Z)", text, re.IGNORECASE | re.DOTALL)
    if not tuntutan:
        tuntutan = re.search(r"Tuntutan Pidana\s*[:.]?\s*(.*)", text, re.IGNORECASE)
    info["Tuntutan Pidana"] = (tuntutan.group(1).replace("\n", " ").strip(" ;") if tuntutan else "Tidak ditemukan")
    # === Hukuman Pidana ===
    putusan = re.search(r"pidana\s+penjara\s+selama\s+(.*?)(?=,|;|\n\d|\Z)", text, re.IGNORECASE | re.DOTALL)
    if not putusan:
        putusan = re.search(r"dibebaskan\s+dari\s+tahanan[^\n;]*", text, re.IGNORECASE)
    if not putusan:
        putusan = re.search(r"Hukuman Pidana\s*[:.]?\s*(.*)", text, re.IGNORECASE)

    info["Hukuman Pidana"] = (
        putusan.group(0).replace("\n", " ").strip(" ;") if putusan else "Tidak ditemukan"
    )
    # === Tanggal Putusan ===
    tanggal = re.search(r"tanggal\s+([\w\s\d]+?\d{4})", text, re.IGNORECASE)
    info["Tanggal Putusan"] = tanggal.group(1).strip(" ,") if tanggal else "Tidak ditemukan"

    # === Hakim Ketua & Hakim Anggota ===
    hakim_ketua, hakim_anggota = None, []

    # normalisasi spasi
    data = " ".join(text.split())

    # ambil Hakim Ketua = sebelum "sebagai Hakim Ketua"
    mengadilis = [
        "MENGADILI:",
        "M E N G A D I L I:",
        "M E N G A D I L I :",
        "M E N G A D I L I"
    ]
    
    # Memulai pencarian setelah kata MENGADI
    start = -1
    
    for i in mengadilis:
        if start != -1:
            break
    
        start = data.find(i)

    mengadili = data[start:]
    mengadili = mengadili[mengadili.find("6."):]
    print(mengadili)
    m_ketua = re.search(r", oleh (?:kami,)?(.*?)\s*,\s*sebagai Hakim Ketua", mengadili, re.IGNORECASE)
    if m_ketua:
        hakim_ketua = m_ketua.group(1).strip(" ,")

    # ambil Hakim Anggota = setelah "sebagai Hakim Ketua" sampai "masing"
    m_anggota = re.search(r"sebagai Hakim Ketua, (.*?) masing", mengadili, re.IGNORECASE)
    if m_anggota:
        bagian = m_anggota.group(1).strip(" ,")
        # split berdasarkan koma atau kata "dan"
        hakim_anggota = bagian.lower().split("dan")
        hakim_anggota = [h.strip(" ,") for h in hakim_anggota if h.strip()]

    # fallback
    if not hakim_ketua:
        ketua = re.search(r"Hakim Ketua\s*[:.]?\s*(.*)", text, re.IGNORECASE)
        hakim_ketua = ketua.group(1).strip(" ,") if ketua else "Tidak ditemukan"

    if not hakim_anggota:
        anggota = re.findall(r"Hakim Anggota\s*\d?\s*[:.]?\s*(.*)", text, re.IGNORECASE)
        hakim_anggota = [a.strip(" ,") for a in anggota] if anggota else ["Tidak ditemukan"]

    # simpan ke dictionary
    info["Hakim Ketua"] = hakim_ketua
    info["Hakim Anggota"] = hakim_anggota



    # === Panitera ===

    panitera = re.search(r"dibantu\s+oleh\s+(.*),\s*Panitera", mengadili, re.IGNORECASE)
    info["Panitera"] = panitera.group(1).strip(" ,") if panitera else "Tidak ditemukan"

    # === Penuntut Umum ===
 
    penuntut = re.search(r"dihadiri\s+oleh\s+(.*),\s*Penuntut Umum", mengadili, re.IGNORECASE)
    info["Penuntut Umum"] = penuntut.group(1).strip(" ,") if penuntut else "Tidak ditemukan"

    return info


def save_to_txt(info, filename="hasil_putusan.txt"):
    with open(filename, "w", encoding="utf-8") as f:
        f.write(f"Nomor Putusan : {info['Nomor Putusan']}\n")

        for i, nama in enumerate(info["Nama Terdakwa"], start=1):
            f.write(f"Terdakwa Ke : {i}\n")
            f.write(f"Nama Terdakwa :  {nama} ;\n\n")

        f.write("TUNTUTAN\n")
        f.write(f"Tindak Pidana : {info['Tindak Pidana']}\n")
        f.write(f"Melanggar KUHP : {info['Melanggar KUHP']}\n")
        f.write(f"Tuntutan Pidana :  {info['Tuntutan Pidana']} ;\n\n")

        f.write("PUTUSAN\n")
        f.write(f"Tindak Pidana :  “{info['Tindak Pidana']}” \n")
        f.write(f"Hukuman Pidana : {info['Hukuman Pidana']} ;\n\n")

        f.write(f"Tanggal Putusan : {info['Tanggal Putusan']}, \n")
        f.write(f"Hakim Ketua : {info['Hakim Ketua']}\n")
        for i, h in enumerate(info["Hakim Anggota"], start=1):
            f.write(f"Hakim Anggota {i} : {h}\n")
        f.write(f"Panitera : {info['Panitera']}\n")
        f.write(f"Penuntut Umum :  {info['Penuntut Umum']}\n")

In [109]:
import os
# from courtHistoryCsv import courtHistory

def main():

    start_path = "./input/"
    hasil_path = "./output/"
    daftarFile = os.listdir(start_path)
    listHasil = []


    for tiapFile in daftarFile:
        print("======= ROW HASIL =======")    
        with open(start_path + tiapFile, "r", encoding="utf-8") as file:
            hasil = extract_detailed_information(file.read())
            save_to_txt(hasil,hasil_path + tiapFile + ".txt")
            (hasil)
        print("======= ROW AKHIR =======")
        print()
 

main()

======= ROW HASIL =======
['MISRO Bin DARSITO; 2. Tempat lahir : Banyumas; 3. Umur / Tanggal lahir : 35 Tahun / 1 Juni 1989; 4. Jenis kelamin : Laki-laki; 5. Kebangsaan : Indonesia; 6. Tempat tinggal : Desa Kramat RT. 005 RW. 003 Kecamatan Kembaran Kabupaten Banyumas; 7. Agama : Islam; 8. Pekerjaan : Buruh Harian Lepas; Terdakwa ditangkap pada tanggal 22 November 2024; Terdakwa ditahan dalam tahanan Rumah Tahanan Negara oleh: 1. Penyidik sejak tanggal 23 November 2024 sampai dengan tanggal 12 Desember 2024; 2. Perpanjangan Penuntut Umum sejak tanggal 13 Desember 2024 sampai dengan tanggal 21 Januari 2025; 3. Perpanjangan Pertama Ketua Pengadilan Negeri sejak tanggal 22 Januari 2025 sampai dengan tanggal 20 Februari 2025; 4. Perpanjangan Kedua Ketua Pengadilan Negeri sejak tanggal 21 Februari 2025 sampai dengan tanggal 22 Maret 2025; 5. Penuntut Umum sejak tanggal 10 Maret 2025 sampai dengan tanggal 29 Maret 2025; 6. Perpanjangan Ketua Pengadilan Negeri sejak tanggal 30 Maret 2025 sampa